In [68]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

In [69]:
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1,low_memory=False)
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
# half_count = len(loans_2007) / 2
# loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
# loans_2007.to_csv('loans_2007.csv', index=False)

In [93]:
loans_2007 = pd.read_csv("loans_2007.csv",low_memory=False)
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## FIRST STEP: FEATURE PREPARATION
#### Looking at the csv file

The csv file contains 52 columns, we are going to examine in groups of 18.

We are going to pay attention to any column that: 

- leak information from the future (after the loan has already been funded):
This information comes from the credit evolution (comes from the future after that the loans is  funded; this information is never available for the investor.
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club).
- formatted poorly and need to be cleaned up.
- require more data or a lot of processing to turn into a useful feature.
- contain redundant information.

------------------------------------------------------------------------
###### FIRST GROUP OF 18 COLUMNS: 


-    id: randomly generated field by Lending Club for unique identification purposes only
-    member_id: also a randomly generated field by Lending Club for unique identification purposes only
-    funded_amnt: leaks data from the future (after the loan is already started to be funded)
-    funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
-    grade: contains redundant information as the interest rate column (int_rate)
-    sub_grade: also contains redundant information as the interest rate column (int_rate)
-    emp_title: requires other data and a lot of processing to potentially be useful
-    issue_d: leaks data from the future (after the loan is already completed funded)
++++++

In [72]:
features_to_drop = ["id","member_id","funded_amnt","funded_amnt_inv","grade","sub_grade","emp_title","issue_d"]

loans_2007.drop(labels=features_to_drop,inplace=True,axis="columns")

###### SECOND GROUP OF 18 COLUMNS LIST: 
-    zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
-    out_prncp: leaks data from the future, (after the loan already started to be paid off)
-    out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
-    total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
-    total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
-    total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

In [73]:
features_to_remove2 = ["zip_code","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp"]

loans_2007.drop(labels=features_to_remove2,axis="columns",inplace=True)

###### THIRD GROUP OF 18 COLUMNS LIST:
-    total_rec_int: leaks data from the future, (after the loan already started to be paid off),
-   total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
-    recoveries: also leaks data from the future, (after the loan already started to be paid off),
-    collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
-    last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
-    last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

In [74]:
features_to_drop3 = ["total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt"]

loans_2007.drop(labels=features_to_drop3,axis="columns",inplace=True)

print(loans_2007.head(0))
print(loans_2007.shape[1])

Empty DataFrame
Columns: [loan_amnt, term, int_rate, installment, emp_length, home_ownership, annual_inc, verification_status, loan_status, pymnt_plan, purpose, title, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, last_credit_pull_d, collections_12_mths_ex_med, policy_code, application_type, acc_now_delinq, chargeoff_within_12_mths, delinq_amnt, pub_rec_bankruptcies, tax_liens]
Index: []

[0 rows x 32 columns]
32


### SELECTING THE TARGET COLUMN: 

- Loan_status it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one

In [75]:
print(loans_2007["loan_status"].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


###### Meaning of the values: 

- Fully Paid:     Loan has been fully paid off.
- Charged Off: 	Loan for which there is no longer a reasonable expectation of further payments.
- Does not meet the credit policy: 	While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.


- Charged Off: While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.


- In Grace Period:	The loan is past due but still in the grace period of 15 days.
- Late (16-30 days):	Loan hasn't been paid in 16 to 30 days (late on the current payment). 
- Late (31-120 days):	Loan hasn't been paid in 31 to 120 days (late on the current payment).
- Current: 	Loan is up to date on current payments.
- Default: 	Loan is defaulted on and no payment has been made for more than 121 days.

<i>Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. You can read about the difference here.

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case.<i/>

In [76]:
loans_2007 = loans_2007.loc[(loans_2007["loan_status"]=="Fully Paid")|(loans_2007["loan_status"]=="Charged Off")]

mapping_dict = {"loan_status":{"Fully Paid":1,"Charged Off":0}}

loans_2007.replace(to_replace=mapping_dict,inplace=True)

#### Let's look for any columns that contain only one unique value and remove them.

In [77]:
drop_columns = []

for column in loans_2007.columns:
    serie = loans_2007[column]
    serie.dropna(inplace=True)
    uniques = serie.unique()
    if len(uniques)<=1:
        drop_columns.append(column)
        
loans_2007.drop(labels=drop_columns,axis=1,inplace=True)

print(drop_columns)

loans = loans_2007.copy()

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [78]:

null_counts = loans.isnull().sum()

print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


###### While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

In [79]:
loans.drop(labels=["pub_rec_bankruptcies"],inplace=True,axis=1)

loans.dropna(how="any",axis=0,inplace=True)

print(loans.dtypes.value_counts())


object     11
float64    10
int64       1
dtype: int64


###### The object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore


In [80]:
object_columns_df = loans.select_dtypes(include=[object])

print(object_columns_df[0:2])

         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   
1   60 months   15.27%   < 1 year           RENT     Source Verified   

       purpose     title addr_state earliest_cr_line revol_util  \
0  credit_card  Computer         AZ         Jan-1985      83.7%   
1          car      bike         GA         Apr-1999       9.4%   

  last_credit_pull_d  
0           Jun-2016  
1           Sep-2013  


##### Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

-    home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
-    verification_status: indicates if income was verified by Lending Club,
-    emp_length: number of years the borrower was employed upon time of application,
-    term: number of payments on the loan, either 36 or 60,
-    addr_state: borrower's state of residence,
-    purpose: a category provided by the borrower for the loan request,
-    title: loan title provided the borrower

<i> <b>Based on the first row's values for purpose and title, it seems like these columns could reflect the same information.
<i/></b>


Some meanings:

- int_rate: interest rate of the loan in %.
- revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit

In [81]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18513
MORTGAGE    17112
OWN          2984
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16696
Verified           12290
Source Verified     9722
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1229
n/a          1032
Name: emp_length, dtype: int64
 36 months    29041
 60 months     9667
Name: term, dtype: int64
CA    6958
NY    3713
FL    2791
TX    2667
NJ    1798
IL    1483
PA    1473
VA    1376
GA    1364
MA    1301
OH    1179
MD    1026
AZ     850
WA     822
CO     770
NC     753
CT     730
MI     712
MO     671
MN     603
NV     481
SC     462
WI     441
AL     437
OR     436
LA     430
KY     315
OK     290
KS     260
UT     254
AR     237
DC     209
RI     196
NM     184
WV     172
NH     166
HI     166
DE     113
MT      83
WY      80
AK      

###### The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering.

In [82]:
# Purpose and title looking the best column to use: 
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    18130
credit_card            5039
other                  3864
home_improvement       2897
major_purchase         2155
small_business         1762
car                    1510
wedding                 929
medical                 680
moving                  576
vacation                375
house                   369
educational             320
renewable_energy        102
Name: purpose, dtype: int64
Debt Consolidation                                  2104
Debt Consolidation Loan                             1632
Personal Loan                                        642
Consolidation                                        494
debt consolidation                                   485
Credit Card Consolidation                            353
Home Improvement                                     346
Debt consolidation                                   324
Small Business Loan                                  310
Credit Card Loan                                     305
Personal  

<i>It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation)


The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.


We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

The addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

    earliest_cr_line: The month the borrower's earliest reported credit line was opened,
    last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.


In [83]:
#We can use the following mapping to clean the emp_length column:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans.drop(labels=["last_credit_pull_d","addr_state","title","earliest_cr_line"],axis=1,inplace=True)

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype(float)

loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype(float)

loans.replace(to_replace=mapping_dict,inplace=True)


In [84]:
#Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy 
#variables so we can use them in our model. We first need to use the Pandas get_dummies.
df_dummies = pd.get_dummies(loans[["home_ownership","verification_status","purpose","term"]])

loans.drop(labels=["home_ownership","verification_status","purpose","term"],axis=1,inplace=True)

loans = pd.concat([loans,df_dummies],axis=1)

In [85]:
#As we prepared the data, we removed columns that had data leakage issues, contained 
#redundant information, or required additional processing to turn into useful features. 
# We cleaned features that had formatting issues, and converted categorical columns to dummy variables.

loans.to_csv("cleaned_loans_2007.csv")

## FINAL STEP: TESTING MODELS

##### Objective: build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not. standpoint of a conservative investor.

- Rmemember: there's a class imbalance in our target column, loan_status. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.
This causes a major issue when we use accuracy as a metric. This is because due to the class imbalance, a classifier can predict 1 for every row

In [86]:
loans = pd.read_csv("cleaned_loans_2007.csv")

print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38708 entries, 0 to 38707
Data columns (total 39 columns):
Unnamed: 0                             38708 non-null int64
loan_amnt                              38708 non-null float64
int_rate                               38708 non-null float64
installment                            38708 non-null float64
emp_length                             38708 non-null int64
annual_inc                             38708 non-null float64
loan_status                            38708 non-null int64
dti                                    38708 non-null float64
delinq_2yrs                            38708 non-null float64
inq_last_6mths                         38708 non-null float64
open_acc                               38708 non-null float64
pub_rec                                38708 non-null float64
revol_bal                              38708 non-null float64
revol_util                             38708 non-null float64
total_acc                    

COUNTING TYPE OF ERROS (REMEMBER): 

tn = ((predictions==0)&(loans["loan_status"]==0)).sum()

tp = ((predictions==1)&(loans["loan_status"]==1)).sum()

fp = ((predictions==1)&(loans["loan_status"]==0)).sum()

fn = ((predictions==0)&(loans["loan_status"]==1)).sum()

#### We established that this is a binary classification problem

#### Error Metrics :
In this case, we're primarily concerned with false positives and false negatives. In the first one we are go to loose money, in the second one we loose the possibilitie of win money. The true positivies and true negatives are the coorect predictions and don't contribute to the error metric. 

<I> conservative investor would want to minimize risk, and avoid false positives as much as possible.

we should optimize for:

-   high recall (true positive rate) --> tpr = tp / (tp + fn)
-   low fall-out (false positive rate) --> fpr = fp / (fp + tn)-


-    False Positive Rate -- "what percentage of my 1 predictions are incorrect?"
 -       In this case, "what percentage of the loans that I fund would not be repaid?"
-  True Positive Rate -- "what percentage of all the possible 1 predictions am I making?"
   -     In this case, "what percentage of loans that could be funded would I fund?"

In [87]:
#Supose that we predict all ones: 
#Predict that all loans will be paid off on time.
predictions = pd.Series(np.ones(loans.shape[0]))

tn = ((predictions==0)&(loans["loan_status"]==0)).sum()

tp = ((predictions==1)&(loans["loan_status"]==1)).sum()

fp = ((predictions==1)&(loans["loan_status"]==0)).sum()

fn = ((predictions==0)&(loans["loan_status"]==1)).sum()

fpr = fp / (fp + tn)

tpr = tp / (tp + fn)

print(fpr,tpr)

1.0 1.0


<i>both metrics are 1: all rows to one, we predict correctly all tp but fn = 0 (because there is no predictions-value 0!
also there is no predictions value for tnSames with fp because there is not 0's!) then both ratios are fp/fp and tp/tp.

Let's use Logistics regressions because: 

  -   it's quick to train and we can iterate more quickly,
  -  it's less prone to overfitting than more complex models like decision trees,
  - it's easy to interpret.

In [88]:
features = loans.drop("loan_status",axis=1)
target = loans["loan_status"]

lr = LogisticRegression()
lr.fit(features,target)
predictions = lr.predict(features)


In [89]:
# In order to get a realistic depiction of the accuracy of the model, let's perform k-fold cross validation. 
# We can use the cross_val_predict() function from the sklearn.model_selection package

lr = LogisticRegression()

predictions = cross_val_predict(lr,features,target,cv=3)

predictions = pd.Series(predictions)

tp = ((predictions==1)&(target==1)).sum()
tn = ((predictions==0)&(target==0)).sum()
fp = ((predictions==1)&(target==0)).sum()
fn = ((predictions==0)&(target==1)).sum()

tpr = tp / (tp + fn)

fpr = fp / (fp + tn)

print(fpr,tpr)

0.970080142476 0.990813767262


##### Theory:

We'll look into oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where loan_status is 0, and where loan_status is 1. This way, the classifier is forced to make actual predictions, since predicting all 1s or all 0s will only result in 50% accuracy at most.

The downside of this technique is that since it has to preserve an equal ratio, you have to either:

    - Throw out many rows of data. If we wanted equal numbers of rows where loan_status is 0 and where loan_status is 1, one way we could do that is to delete rows where loan_status is 1.
    - Copy rows multiple times. One way to equalize the 0s and 1s is to copy rows where loan_status is 0.
    - Generate fake data. One way to equalize the 0s and 1s is to generate new rows where loan_status is 0.
    
Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn.

We can do this by setting the class_weight parameter to balanced when creating the LogisticRegression instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

--------------------------------------------
###### By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. You can read more about the parameter here. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

In [90]:
#Again with data-set balanced

lr = LogisticRegression(class_weight = "balanced")

predictions = cross_val_predict(lr,features,target,cv=3)

predictions = pd.Series(predictions)

tp = ((predictions==1)&(target==1)).sum()
tn = ((predictions==0)&(target==0)).sum()
fp = ((predictions==1)&(target==0)).sum()
fn = ((predictions==0)&(target==1)).sum()

tpr = tp / (tp + fn)

fpr = fp / (fp + tn)

print(fpr,tpr)


0.409082813891 0.632490254737


###### We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. Our true positive rate is now around 67%, and our false positive rate is around 40%.
###### We want to reduce more the fpr, then we can assign manually the penalties: 

In [91]:
#Again with manual penalties:

penalty = {0:10,1:1}

lr = LogisticRegression(class_weight = "balanced")

predictions = cross_val_predict(lr,features,target,cv=3)

predictions = pd.Series(predictions)

tp = ((predictions==1)&(target==1)).sum()
tn = ((predictions==0)&(target==0)).sum()
fp = ((predictions==1)&(target==0)).sum()
fn = ((predictions==0)&(target==1)).sum()

tpr = tp / (tp + fn)

fpr = fp / (fp + tn)

print(fpr,tpr)

0.409082813891 0.632490254737


###### It looks like assigning manual penalties lowered the false positive rate to 7%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. 

-------------------------------------------------
### TRYING ANOTHER MODEL:
###### Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. 

In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict

rf = RandomForestClassifier(random_state=1,class_weight="balanced")

predictions = cross_val_predict(rf,features,target,cv=3)

predictions = pd.Series(predictions)

tp = ((predictions==1)&(target==1)).sum()
tn = ((predictions==0)&(target==0)).sum()
fp = ((predictions==1)&(target==0)).sum()
fn = ((predictions==0)&(target==1)).sum()

tpr = tp / (tp + fn)

fpr = fp / (fp + tn)

print(fpr,tpr)

0.636153161175 0.648777687124


#### CONCLUSIONS:
<i><r>
Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of 7%, and a true positive rate of 20%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 20% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

    We can tweak the penalties further.
    We can try models other than a random forest and logistic regression.
    We can use some of the columns we discarded to generate better features.
    We can ensemble multiple models to get more accurate predictions.
    We can tune the parameters of the algorithm to achieve higher performance.
